In [6]:
import pandas as pd
import chess

In [7]:
df_chess = pd.read_csv('..\chess_games.csv')

In [21]:
board = chess.Board()
for move in df_chess['moves'][3].split(" "):
    board.push_san(move)
print(board.fen())
print(board)

7r/R5Qp/2p3Nk/3pP1p1/8/4P1P1/1P2BP1P/5RK1 b - - 0 31
. . . . . . . r
R . . . . . Q p
. . p . . . N k
. . . p P . p .
. . . . . . . .
. . . . P . P .
. P . . B P . P
. . . . . R K .
